In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from shapely.geometry import shape
from shapely.geometry import Polygon

In [ ]:
# Leer los datos desde el archivo .parquet
file_path = 'quejas-final.parquet'  # Reemplaza con la ruta correcta a tu archivo
df = pd.read_parquet(file_path)

# Filtrar filas con valores None en la columna 'geo_shape'
df = df[df['geo_shape'].notna()]

def convert_and_simplify(geojson_str, tolerance=0.001):
    geom = shape(json.loads(geojson_str))
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance)
    else:
        return geom

df['geometry'] = df['geo_shape'].apply(lambda x: convert_and_simplify(x))

# Convertir las geometrías de 'geo_shape' al formato adecuado

# Crear un GeoDataFrame con las geometrías
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Agrupar por distrito y contar el número de solicitudes
conteo_barrio = df['barrio_localización'].value_counts().reset_index()
conteo_barrio.columns = ['barrio_localización', 'count']

# Calcular la densidad de solicitudes por área (esto requiere tener un campo 'Shape_Area' en tu dataset)
conteo_barrio = conteo_barrio.merge(df[['barrio_localización', 'Shape_Area']].drop_duplicates(), on='barrio_localización')
conteo_barrio['density'] = conteo_barrio['count'] / conteo_barrio['Shape_Area']

# Crear un mapa base usando las geometrías
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=['Application Density per Neighbourhood'],
    specs=[[{"type": "mapbox"}]]
)

# Añadir el trazo para la densidad
density_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=conteo_barrio['barrio_localización'],
    z=conteo_barrio['density'],
    colorscale='YlGnBu',
    colorbar_title="Densidad",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=True
)

# Añadir el trazo para el conteo
count_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('barrio_localización')['geometry'].__geo_interface__,
    locations=conteo_barrio['barrio_localización'],
    z=conteo_barrio['count'],
    colorscale='YlOrRd',
    colorbar_title="Conteo",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False
)

fig.add_trace(density_trace, row=1, col=1)
fig.add_trace(count_trace, row=1, col=1)

# Configuración del mapa de Valencia
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=11,
    mapbox_center={"lat": 39.4699, "lon": -0.3763},  # Coordenadas para centrar el mapa en Valencia
    title_text="Application Density per Neighbourhood"
)

# Añadir el menú de selección
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [{"visible": [True, False]}],
                    "label": "Density",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, True]}],
                    "label": "Count",
                    "method": "update"
                }
            ],
            "direction": "down",
            "pad": {"r": 10, "t": 10},
            "showactive": True,
            "x": 0,
            "xanchor": "left",
            "y": 1.2,
            "yanchor": "top"
        }
    ]
)

# Mostrar el gráfico
fig.show()


In [ ]:
# Leer los datos desde el archivo .parquet
file_path = 'quejas-final.parquet'  # Reemplaza con la ruta correcta a tu archivo
df = pd.read_parquet(file_path)

# Filtrar filas con valores None en la columna 'geo_shape'

# Convertir las geometrías de 'geo_shape' al formato adecuado
def convert_and_simplify(geojson_str, tolerance=0.001):
    geom = shape(json.loads(geojson_str))
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance)
    else:
        return geom

df['geometry'] = df['geo_shape_dis'].apply(lambda x: convert_and_simplify(x))

# Crear un GeoDataFrame con las geometrías
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Agrupar por distrito y contar el número de solicitudes
conteo_barrio = df['distrito_localización'].value_counts().reset_index()
conteo_barrio.columns = ['distrito_localización', 'count']

# Calcular la densidad de solicitudes por área (esto requiere tener un campo 'Shape_Area' en tu dataset)
conteo_barrio = conteo_barrio.merge(df[['distrito_localización', 'Shape_Area']].drop_duplicates(), on='distrito_localización')
conteo_barrio['density'] = conteo_barrio['count'] / conteo_barrio['Shape_Area']

# Crear un mapa base usando las geometrías
fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=['Application Density per District'],
    specs=[[{"type": "mapbox"}]]
)

# Añadir el trazo para la densidad
density_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
    locations=conteo_barrio['distrito_localización'],
    z=conteo_barrio['density'],
    colorscale='YlGnBu',
    colorbar_title="Densidad",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=True
)

# Añadir el trazo para el conteo
count_trace = go.Choroplethmapbox(
    geojson=gdf.set_index('distrito_localización')['geometry'].__geo_interface__,
    locations=conteo_barrio['distrito_localización'],
    z=conteo_barrio['count'],
    colorscale='YlOrRd',
    colorbar_title="Conteo",
    marker_line_width=1,
    marker_opacity=0.7,
    visible=False
)

fig.add_trace(density_trace, row=1, col=1)
fig.add_trace(count_trace, row=1, col=1)

# Configuración del mapa de Valencia
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=11,
    mapbox_center={"lat": 39.4699, "lon": -0.3763},  # Coordenadas para centrar el mapa en Valencia
    title_text="Application Density per District"
)

# Añadir el menú de selección
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [{"visible": [True, False]}],
                    "label": "Density",
                    "method": "update"
                },
                {
                    "args": [{"visible": [False, True]}],
                    "label": "Count",
                    "method": "update"
                }
            ],
            "direction": "down",
            "pad": {"r": 10, "t": 10},
            "showactive": True,
            "x": 0,
            "xanchor": "left",
            "y": 1.2,
            "yanchor": "top"
        }
    ]
)

# Mostrar el gráfico
fig.show()
